In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# LinearBlockGS

LinearBlockGS uses Block Gauss-Seidel to solve the linear system. LinearBlockGS iterates until the linear
residual is below a tolerance, or the maximum number of iterations has been exceeded. As such,
it is generally usable for any system topology, and can handle cycles and implicit states
alike. It is not always the best solver to choose, however, and is known to diverge or plateau
on some problems. In such a case, you may need to use a solver such as
[ScipyKrylov](../../../_srcdocs/packages/solvers.linear/scipy_iter_solver).

LinearBlockGS is a block solver, so you can specify different linear solvers in the subsystems and they
will be utilized to solve the subsystem linear problem.

Note that systems without cycles or implicit states will converge in one iteration of Block Gauss-Seidel.

Here, we calculate the total derivatives across the Sellar system.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src24", get_code("openmdao.test_suite.components.sellar.SellarDis1withDerivatives"), display=False)

:::{Admonition} `SellarDis1withDerivatives` class definition 
:class: dropdown

{glue:}`code_src24`
:::

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src25", get_code("openmdao.test_suite.components.sellar.SellarDis2withDerivatives"), display=False)

:::{Admonition} `SellarDis2withDerivatives` class definition 
:class: dropdown

{glue:}`code_src25`
:::

In [ ]:
import numpy as np
import openmdao.api as om

from openmdao.test_suite.components.sellar import SellarDis1withDerivatives, SellarDis2withDerivatives

prob = om.Problem()
model = prob.model

model.add_subsystem('d1', SellarDis1withDerivatives(), promotes=['x', 'z', 'y1', 'y2'])
model.add_subsystem('d2', SellarDis2withDerivatives(), promotes=['z', 'y1', 'y2'])

model.add_subsystem('obj_cmp', om.ExecComp('obj = x**2 + z[1] + y1 + exp(-y2)',
                                           z=np.array([0.0, 0.0]), x=0.0),
                    promotes=['obj', 'x', 'z', 'y1', 'y2'])

model.add_subsystem('con_cmp1', om.ExecComp('con1 = 3.16 - y1'), promotes=['con1', 'y1'])
model.add_subsystem('con_cmp2', om.ExecComp('con2 = y2 - 24.0'), promotes=['con2', 'y2'])

model.linear_solver = om.LinearBlockGS()
model.nonlinear_solver = om.NonlinearBlockGS()

prob.setup()

prob.set_val('x', 1.)
prob.set_val('z', np.array([5.0, 2.0]))

prob.run_model()

wrt = ['z']
of = ['obj']

J = prob.compute_totals(of=of, wrt=wrt, return_format='flat_dict')

In [ ]:
print(J['obj', 'z'][0][0])
print(J['obj', 'z'][0][1])

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(J['obj', 'z'][0][0], 9.61001056, .00001)
assert_near_equal(J['obj', 'z'][0][1], 1.78448534, .00001)

## LinearBlockGS Options

In [ ]:
om.show_options_table("openmdao.solvers.linear.linear_block_gs.LinearBlockGS")

## LinearBlockGS Constructor

The call signature for the `LinearBlockGS` constructor is:

```{eval-rst}
    .. automethod:: openmdao.solvers.linear.linear_block_gs.LinearBlockGS.__init__
        :noindex:
```

## Aitken relaxation

This solver implements Aitken relaxation, as described in Algorithm 1 of this paper on aerostructual design [optimization](https://mdolab.engin.umich.edu/bibliography/Kenway2014a). The relaxation is turned off by default, but it may help convergence for more tightly coupled models.


## LinearBlockGS Option Examples

**maxiter**

  This lets you specify the maximum number of Gauss-Seidel iterations to apply. In this example, we
  cut it back from the default, ten, down to two, so that it terminates a few iterations earlier and doesn't
  reach either of the specified absolute or relative tolerances.

In [ ]:
from openmdao.test_suite.components.sellar import SellarDis1withDerivatives, SellarDis2withDerivatives

prob = om.Problem()
model = prob.model

model.add_subsystem('d1', SellarDis1withDerivatives(), promotes=['x', 'z', 'y1', 'y2'])
model.add_subsystem('d2', SellarDis2withDerivatives(), promotes=['z', 'y1', 'y2'])

model.add_subsystem('obj_cmp', om.ExecComp('obj = x**2 + z[1] + y1 + exp(-y2)',
                                           z=np.array([0.0, 0.0]), x=0.0),
                    promotes=['obj', 'x', 'z', 'y1', 'y2'])

model.add_subsystem('con_cmp1', om.ExecComp('con1 = 3.16 - y1'), promotes=['con1', 'y1'])
model.add_subsystem('con_cmp2', om.ExecComp('con2 = y2 - 24.0'), promotes=['con2', 'y2'])

model.nonlinear_solver = om.NonlinearBlockGS()

model.linear_solver = om.LinearBlockGS()
model.linear_solver.options['maxiter'] = 2

prob.setup()

prob.set_val('x', 1.)
prob.set_val('z', np.array([5.0, 2.0]))

prob.run_model()

wrt = ['z']
of = ['obj']

J = prob.compute_totals(of=of, wrt=wrt, return_format='flat_dict')

In [ ]:
print(J['obj', 'z'][0][0])
print(J['obj', 'z'][0][1])

In [ ]:
assert_near_equal(J['obj', 'z'][0][0], 9.60230118004, .00001)
assert_near_equal(J['obj', 'z'][0][1], 1.78022500547, .00001)

**atol**

  Here, we set the absolute tolerance to a looser value that will trigger an earlier termination. After
  each iteration, the norm of the linear residuals is calculated by calling `apply_linear`. If this norm value is lower than the absolute tolerance `atol`, the iteration will terminate.

In [ ]:
from openmdao.test_suite.components.sellar import SellarDis1withDerivatives, SellarDis2withDerivatives

prob = om.Problem()
model = prob.model

model.add_subsystem('d1', SellarDis1withDerivatives(), promotes=['x', 'z', 'y1', 'y2'])
model.add_subsystem('d2', SellarDis2withDerivatives(), promotes=['z', 'y1', 'y2'])

model.add_subsystem('obj_cmp', om.ExecComp('obj = x**2 + z[1] + y1 + exp(-y2)',
                                           z=np.array([0.0, 0.0]), x=0.0),
                    promotes=['obj', 'x', 'z', 'y1', 'y2'])

model.add_subsystem('con_cmp1', om.ExecComp('con1 = 3.16 - y1'), promotes=['con1', 'y1'])
model.add_subsystem('con_cmp2', om.ExecComp('con2 = y2 - 24.0'), promotes=['con2', 'y2'])

model.nonlinear_solver = om.NonlinearBlockGS()

model.linear_solver = om.LinearBlockGS()
model.linear_solver.options['atol'] = 1.0e-3

prob.setup()

prob.set_val('x', 1.)
prob.set_val('z', np.array([5.0, 2.0]))

prob.run_model()

wrt = ['z']
of = ['obj']

J = prob.compute_totals(of=of, wrt=wrt, return_format='flat_dict')

In [ ]:
print(J['obj', 'z'][0][0])
print(J['obj', 'z'][0][1])

In [ ]:
assert_near_equal(J['obj', 'z'][0][0], 9.61016296175, .00001)
assert_near_equal(J['obj', 'z'][0][1], 1.78456955704, .00001)

**rtol**

  Here, we set the relative tolerance to a looser value that will trigger an earlier termination. After
  each iteration, the norm of the linear residuals is calculated by calling `apply_linear`. If the ratio of the currently calculated norm to the
  initial residual norm is lower than the relative tolerance `rtol`, the iteration will terminate.

In [ ]:
from openmdao.test_suite.components.sellar import SellarDis1withDerivatives, SellarDis2withDerivatives

prob = om.Problem()
model = prob.model

model.add_subsystem('d1', SellarDis1withDerivatives(), promotes=['x', 'z', 'y1', 'y2'])
model.add_subsystem('d2', SellarDis2withDerivatives(), promotes=['z', 'y1', 'y2'])

model.add_subsystem('obj_cmp', om.ExecComp('obj = x**2 + z[1] + y1 + exp(-y2)',
                                           z=np.array([0.0, 0.0]), x=0.0),
                    promotes=['obj', 'x', 'z', 'y1', 'y2'])

model.add_subsystem('con_cmp1', om.ExecComp('con1 = 3.16 - y1'), promotes=['con1', 'y1'])
model.add_subsystem('con_cmp2', om.ExecComp('con2 = y2 - 24.0'), promotes=['con2', 'y2'])

model.nonlinear_solver = om.NonlinearBlockGS()

model.linear_solver = om.LinearBlockGS()
model.linear_solver.options['rtol'] = 1.0e-3

prob.setup()

prob.set_val('x', 1.)
prob.set_val('z', np.array([5.0, 2.0]))

prob.run_model()

wrt = ['z']
of = ['obj']

J = prob.compute_totals(of=of, wrt=wrt, return_format='flat_dict')

In [ ]:
print(J['obj', 'z'][0][0])
print(J['obj', 'z'][0][1])

In [ ]:
assert_near_equal(J['obj', 'z'][0][0], 9.61016296175, .00001)
assert_near_equal(J['obj', 'z'][0][1], 1.78456955704, .00001)